In [6]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from sklearn.utils import resample

# Create a connection with SQL server to get data.
def exec_sql_query(procedure, *param):
    
    from sqlalchemy import create_engine
    import urllib
    params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=LAPTOP-C3LFVOFI;DATABASE=student_life;UID=student_sense;PWD=abhinav123")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    connection = engine.raw_connection()
    
    try:
        cursor = connection.cursor()
        if(param):
            cursor.execute(procedure, [param[0]])
        else : 
            cursor.execute(procedure)
            
        results = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        df = pd.DataFrame.from_records(results, columns=columns)
        cursor.close()
        connection.commit()
    finally:
        connection.close()
    
    del engine
    return df

In [7]:
def get_training_data():
    # Function to return data from tables.
    # returns train_x and train_y.

    # Getting details for all students.
    training_data = exec_sql_query("Exec get_train_x_new")
    # display(training_data.head())

    # Dropping NULL columns as they are not usefull and setting index to DATE.
    row_count = len(training_data)
    training_data_n_null = training_data.dropna(axis=1, how="all")
    training_data_n_null.set_index("DATE", inplace=True)
    training_data_n_null.fillna(0, inplace=True)
    # display(training_data_n_null.head())
    
    # splitting int train_x and train_y
    train_x = training_data_n_null.iloc[:,:-3]
    train_y = training_data_n_null.iloc[:,-3:]
    
    return train_x, train_y

In [8]:
# Returning Stratafied training and test data.

from sklearn.model_selection import train_test_split

def get_split_train_data(x=None, y=None, test_size=0.30, random_state=100, balance_data=True):

    # fucntion to return training and test data sthrough splitting
    if not x and not y:
        x, y = get_training_data()
    
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=test_size, random_state=random_state, shuffle=True)
    
    return train_x, test_x, train_y, test_y

In [11]:
def balance_data(train_x, train_y):
    
    # Converting to Data Frame
    train_df = pd.concat([train_x, train_y], axis=1)
    # Separating in minority and majority class.
    dominant_class = train_df.iloc[:,-1].value_counts().idxmax()
    
    # now we resamples for every class. making the value equal to that of the dominant class.
    classes = list(pd.unique(train_y))
    classes.remove(dominant_class)
    major_df = train_df[train_df.iloc[:,-1] == dominant_class]
    major_class_count = len(major_df)
    result_df = major_df.copy()
    
    for c in classes:
        minor_df = train_df[train_df.iloc[:,-1] == c]
        resampled_minor = resample(minor_df, replace=True, n_samples=major_class_count, random_state=123)
        result_df = result_df.append(resampled_minor, ignore_index=True)
    
    return result_df.iloc[:,:-1] ,result_df.iloc[:,-1] 
     
    
    